In [1]:
import hvplot.pandas
import pandas as pd
import requests

from api_keys import geoapify_key

In [2]:
city_data = pd.read_csv('../WeatherPy.ipynb/output_data/cities.csv')
city_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,al qadarif,14.0349,35.3834,27.51,37,0,4.11,SD,1730932466
1,1,klyuchi,52.2667,79.1667,1.64,50,46,11.16,RU,1730932468
2,2,tiksi,71.6872,128.8694,-13.51,79,100,5.62,RU,1730932469
3,3,olonkinbyen,70.9221,-8.7187,-2.15,80,41,12.36,SJ,1730932470
4,4,petropavlovsk-kamchatsky,53.0452,158.6483,-7.67,49,0,2.00,RU,1730932471


In [4]:
map_plot = city_data.hvplot.points(x="Lng", y="Lat", geo=True, size="Humidity", 
                                   frame_width=800, frame_height=600, hover_cols=['City', 'Humidity'])
map_plot


:Points   [Lng,Lat]   (Humidity,City)

In [5]:
ideal_weather_data = city_data[(city_data['Max Temp'] >22) & (city_data['Wind Speed'] <8) & 
                            (city_data['Cloudiness'] <10) & (city_data['Humidity'] <50)]
ideal_weather_data = ideal_weather_data.dropna()
ideal_weather_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,al qadarif,14.0349,35.3834,27.51,37,0,4.11,SD,1730932466
28,28,kidal,18.4411,1.4078,22.03,20,0,4.91,ML,1730932497
108,108,stephenville,32.2207,-98.2023,23.07,40,0,4.12,US,1730932589
165,165,umm kaddadah,13.6017,26.6876,22.85,23,2,6.96,SD,1730932657
167,167,hualahuises,24.8847,-99.6733,28.32,42,0,1.96,MX,1730932659
221,221,laguna,38.4210,-121.4238,22.28,22,0,6.17,US,1730932720
245,245,kone,-21.0595,164.8658,29.96,38,6,1.23,NC,1730932747
292,292,niafunke,15.9322,-3.9906,27.05,15,0,4.78,ML,1730932799
333,333,post,33.1909,-101.3782,22.76,34,0,2.51,US,1730932848
338,338,center,31.5107,-96.4247,25.22,42,0,3.60,US,1730932854


In [6]:
hotel_df = city_data[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df['Hotel Name']=""
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,al qadarif,SD,14.0349,35.3834,37,
1,klyuchi,RU,52.2667,79.1667,50,
2,tiksi,RU,71.6872,128.8694,79,
3,olonkinbyen,SJ,70.9221,-8.7187,80,
4,petropavlovsk-kamchatsky,RU,53.0452,158.6483,49,
...,...,...,...,...,...,...
543,wilson,US,36.1501,-86.2833,85,
544,flekkefjord,NO,58.2971,6.6607,93,
545,asmar,AF,35.0333,71.3581,46,
546,bulandshahr,IN,28.4000,77.8500,48,


In [8]:
radius = 10000
# categories = "accommodation.hotel"
limit = 20


params ={
    "limit": limit,
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

print("Starting hotel search")

for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    params['filter'] = f"circle:{longitude},{latitude},{radius}"
    params['bias'] = f"proximity:{longitude},{latitude}"
    

    
    base_url = "https://api.geoapify.com/v2/places"
    
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    
    try: 
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except:
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}") 

hotel_df
        

Starting hotel search
al qadarif - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
tiksi - nearest hotel: Арктика
olonkinbyen - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
worcester - nearest hotel: Quality Inn & Suites Worcester
tura - nearest hotel: Hotel Polo Orchid
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
long beach - nearest hotel: Westin
grytviken - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
kemer - nearest hotel: Himeros life
hawaiian paradise park - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
yamada - nearest hotel: No hotel found
metlili chaamba - nearest hotel: No hotel found
lensk - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
berdigestyakh - nearest hotel: No hotel found
khovd - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
0,al qadarif,SD,14.0349,35.3834,37,No hotel found
1,klyuchi,RU,52.2667,79.1667,50,No hotel found
2,tiksi,RU,71.6872,128.8694,79,Арктика
3,olonkinbyen,SJ,70.9221,-8.7187,80,No hotel found
4,petropavlovsk-kamchatsky,RU,53.0452,158.6483,49,ООО Постоялый двор
...,...,...,...,...,...,...
543,wilson,US,36.1501,-86.2833,85,Comfort Suites
544,flekkefjord,NO,58.2971,6.6607,93,Grand hotell Flekkefjord
545,asmar,AF,35.0333,71.3581,46,No hotel found
546,bulandshahr,IN,28.4000,77.8500,48,No hotel found


In [9]:
hotel_map_plot = hotel_df.hvplot.points(x="Lng", y="Lat", geo=True, size="Humidity", 
                                   frame_width=800, frame_height=600, hover_cols=['City', 'Humidity','Hotel', 'Country'])
hotel_map_plot

:Points   [Lng,Lat]   (Humidity,City,Country)